In [4]:
import findspark 
findspark.init()

import pyspark
#Per usare i dataframe si usa SparkSession contenuto in pyspark.sql
from pyspark.sql import SparkSession #SparkContext è contentuto in SparkSession
from pyspark import SparkContext #usato successivamente per creare il RDD


#TRA RDD E DataFrame ci sono due generazioni diverse
spark=SparkSession.builder.getOrCreate()
dati=[('Mario','Rossi',1.7,43,'1999-12-25','M'),
('Carla','Verdi',1.5,54,'1990-08-09','F'),
('Marta','Bianchi',1.75,82,'2000-11-28','F'),
('Pippo','Neri',1.65,77,'2001-10-07','M')]

#creo uno sparkContext
sc=spark.sparkContext

rdd=sc.parallelize(dati)
#Creare un DataFrame da un RDD specificando:
    #RDD di partenza
    #Nome colonne
df=spark.createDataFrame(rdd,['nome','cognome',
                                    'altezza','peso','nascita','sesso'])
df.show()
print("Filtro -> df.filter(df['sesso']=='F').show()")
df.filter(df['sesso']=='F').show()
#OPPURE
print("Filtro alternativo -> df.sesso=='F').show()")
df.filter(df.sesso=='F').show()

print("Conteggia per categoria -> groupBy('sesso').count().show()")
df.groupBy('sesso').count().show()


+-----+-------+-------+----+----------+-----+
| nome|cognome|altezza|peso|   nascita|sesso|
+-----+-------+-------+----+----------+-----+
|Mario|  Rossi|    1.7|  43|1999-12-25|    M|
|Carla|  Verdi|    1.5|  54|1990-08-09|    F|
|Marta|Bianchi|   1.75|  82|2000-11-28|    F|
|Pippo|   Neri|   1.65|  77|2001-10-07|    M|
+-----+-------+-------+----+----------+-----+

Filtro -> df.filter(df['sesso']=='F').show()
+-----+-------+-------+----+----------+-----+
| nome|cognome|altezza|peso|   nascita|sesso|
+-----+-------+-------+----+----------+-----+
|Carla|  Verdi|    1.5|  54|1990-08-09|    F|
|Marta|Bianchi|   1.75|  82|2000-11-28|    F|
+-----+-------+-------+----+----------+-----+

Filtro alternativo -> df.sesso=='F').show()
+-----+-------+-------+----+----------+-----+
| nome|cognome|altezza|peso|   nascita|sesso|
+-----+-------+-------+----+----------+-----+
|Carla|  Verdi|    1.5|  54|1990-08-09|    F|
|Marta|Bianchi|   1.75|  82|2000-11-28|    F|
+-----+-------+-------+----+-------

AGGREGAZIONI PERSONALIZZATE

In [2]:
import findspark 
findspark.init()

import pyspark
#Per usare i dataframe si usa SparkSession contenuto in pyspark.sql
from pyspark.sql import SparkSession #SparkContext è contentuto in SparkSession
from pyspark import SparkContext #usato successivamente per creare il RDD

from pyspark.sql import functions as F 

#TRA RDD E DataFrame ci sono due generazioni diverse
spark=SparkSession.builder.getOrCreate()
dati=[('Mario','Rossi',1.7,43,'1999-12-25','M'),
('Carla','Verdi',1.5,54,'1990-08-09','F'),
('Marta','Bianchi',1.75,82,'2000-11-28','F'),
('Pippo','Neri',1.65,77,'2001-10-07','M')]

#creo uno sparkContext
sc=spark.sparkContext

rdd=sc.parallelize(dati)
#Creare un DataFrame da un RDD specificando:
    #RDD di partenza
    #Nome colonne
df=spark.createDataFrame(rdd,['nome','cognome',
                                    'altezza','peso','nascita','sesso'])
df.show()



print("AGGREGAZIONI PERSONALIZZATE -> agg(F....) + groupBy")
#Per usare funzioni personalizzate per l'aggregazione va importato questo

print("Peso e e altezza media per sesso -> agg(F.mean('peso'),F.mean('altezza')).show()")
#Mostra il peso medio e altezza media
    #groupBy -> raggruppa per sesso
    #agg(..)  -> contiene delle funzioni personalizzate
    #F -> è l'alias per specificare funzioni
df.groupBy('sesso').agg(F.mean('peso'),F.mean('altezza')).show()


print("Peso e e altezza media per sesso arrotondati -> agg(round(F...),2)")
# round arrotonda i numeri
    #2 -> fino alla seconda cifra decimale
df.groupBy('sesso').agg(F.round(F.mean('peso'),2),
                        F.round(F.mean('altezza'),2)).show()

print("ALIAS -> agg(...(F...)).alias(\'NOME\')")
#dopo aver fatto la media da un ALIAS
# ...alias('NUOVO NOME')
df.groupBy('sesso').agg(F.round(F.mean('peso'),2).alias('peso medio'),
                        F.round(F.mean('altezza'),2).alias('altezza media')).show()

+-----+-------+-------+----+----------+-----+
| nome|cognome|altezza|peso|   nascita|sesso|
+-----+-------+-------+----+----------+-----+
|Mario|  Rossi|    1.7|  43|1999-12-25|    M|
|Carla|  Verdi|    1.5|  54|1990-08-09|    F|
|Marta|Bianchi|   1.75|  82|2000-11-28|    F|
|Pippo|   Neri|   1.65|  77|2001-10-07|    M|
+-----+-------+-------+----+----------+-----+

AGGREGAZIONI PERSONALIZZATE -> agg(F....) + groupBy
Peso e e altezza media per sesso -> agg(F.mean('peso'),F.mean('altezza')).show()
+-----+---------+------------------+
|sesso|avg(peso)|      avg(altezza)|
+-----+---------+------------------+
|    M|     60.0|1.6749999999999998|
|    F|     68.0|             1.625|
+-----+---------+------------------+

Peso e e altezza media per sesso arrotondati -> agg(round(F...),2)
+-----+-------------------+----------------------+
|sesso|round(avg(peso), 2)|round(avg(altezza), 2)|
+-----+-------------------+----------------------+
|    M|               60.0|                  1.67|
|   